In [ ]:
import numpy as np
import pandas as pd

In [ ]:
click_ad_df = pd.read_csv('data-files/click_add_sampled.csv')

In [ ]:
click_ad_df.info()

In [ ]:
click_ad_df.head()

In [ ]:
X = click_ad_df.drop(['click', 'id', 'hour', 'device_id', 'device_ip'], axis=1)
y = click_ad_df['click']

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = ohe.fit_transform(X_train)

X_test_encoded = ohe.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

parameters = {'max_depth': [3, 10, None]}

random_forest = RandomForestClassifier(n_estimators=100, criterion='gini', min_samples_split=30, n_jobs=-1)

grid_search = GridSearchCV(random_forest, parameters, n_jobs=-1, cv=3, scoring='roc_auc')
grid_search.fit(X_train_encoded, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

random_forest_best = grid_search.best_estimator_
pos_prob = random_forest_best.predict_proba(X_test_encoded)[:, 1]
print('ROC AUC : {0}'.format(roc_auc_score(y_test, pos_prob)))